<a href="https://colab.research.google.com/github/Valentina-Fontanarosa/wellness-forecast-rs/blob/main/Modello_previsione_benessere_giornaliero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy pandas scipy matplotlib seaborn bokeh statsmodels scikit-learn utilsforecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.1 MB/s eta 0:00:00
